In [1]:
import sys
from pathlib import Path
import pandas as pd

# Add the parent directory to Python path
project_root = Path('..')
sys.path.append(str(project_root))
output_dir   = project_root / 'Output'
analysis_dir = project_root / 'Analysis'

# Import the module directly
import Code.DataUtils as du

In [2]:
# Caminhos para os arquivos gerados
path_cosif_final           = output_dir / 'df_cosif_final.parquet'
path_ifdata_final          = output_dir / 'df_ifdata_final.parquet'
path_map_prud_geral        = output_dir / 'df_mapeamento_prudencial_geral.parquet'
path_if_cadastro_proc      = output_dir / 'df_if_cadastro_completo_processado.parquet' # NOVO ARQUIVO
path_banks_csv             = analysis_dir / 'Banks.csv'

print(f"\nPasta Output: {output_dir}")
print(f"Pasta Analysis: {analysis_dir}")
print("-" * 50)


Pasta Output: ..\Output
Pasta Analysis: ..\Analysis
--------------------------------------------------


In [3]:
# ===== CÉLULA 2: Carregamento dos DataFrames =====
print("\n--- Carregando DataFrames ---")
df_cosif, df_ifdata, df_map_prud, df_banks, df_if_cadastro_proc = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
dataframes_loaded = True

try:
    df_cosif = pd.read_parquet(path_cosif_final)
    print(f"df_cosif_final carregado. Shape: {df_cosif.shape}")
except FileNotFoundError:
    print(f"ERRO: {path_cosif_final} não encontrado."); dataframes_loaded = False
except Exception as e:
    print(f"Erro ao carregar {path_cosif_final}: {e}"); dataframes_loaded = False


try:
    df_ifdata = pd.read_parquet(path_ifdata_final)
    print(f"df_ifdata_final carregado. Shape: {df_ifdata.shape}")
except FileNotFoundError:
    print(f"ERRO: {path_ifdata_final} não encontrado."); dataframes_loaded = False
except Exception as e:
    print(f"Erro ao carregar {path_ifdata_final}: {e}"); dataframes_loaded = False

try:
    df_map_prud = pd.read_parquet(path_map_prud_geral)
    print(f"df_mapeamento_prudencial_geral carregado. Shape: {df_map_prud.shape}")
except FileNotFoundError:
    print(f"ERRO: {path_map_prud_geral} não encontrado."); dataframes_loaded = False
except Exception as e:
    print(f"Erro ao carregar {path_map_prud_geral}: {e}"); dataframes_loaded = False

try:
    df_if_cadastro_proc = pd.read_parquet(path_if_cadastro_proc)
    print(f"df_if_cadastro_completo_processado carregado. Shape: {df_if_cadastro_proc.shape}")
except FileNotFoundError:
    print(f"ERRO: {path_if_cadastro_proc} não encontrado."); dataframes_loaded = False # Não é estritamente essencial para tudo, mas para Nubank sim
except Exception as e:
    print(f"Erro ao carregar {path_if_cadastro_proc}: {e}"); dataframes_loaded = False


try:
    df_banks = pd.read_csv(path_banks_csv, sep=';', dtype=str)
    if 'CNPJ' in df_banks.columns and hasattr(du, 'standardize_cnpj_base8'):
        df_banks['CNPJ_ORIGINAL'] = df_banks['CNPJ']
        df_banks['CNPJ'] = du.standardize_cnpj_base8(df_banks['CNPJ']) # Usa a função do módulo
    print(f"df_banks carregado. Shape: {df_banks.shape}")
except FileNotFoundError:
    print(f"ERRO: {path_banks_csv} não encontrado."); dataframes_loaded = False
except Exception as e:
    print(f"Erro ao carregar {path_banks_csv}: {e}"); dataframes_loaded = False


if not dataframes_loaded:
    print("\nAVISO IMPORTANTE: Um ou mais DataFrames essenciais não puderam ser carregados. As análises podem falhar ou ser incompletas.")
print("-" * 50)


--- Carregando DataFrames ---
df_cosif_final carregado. Shape: (224757, 11)
df_ifdata_final carregado. Shape: (2907128, 27)
df_mapeamento_prudencial_geral carregado. Shape: (162, 3)
df_if_cadastro_completo_processado carregado. Shape: (65388, 18)
df_banks carregado. Shape: (42, 3)
--------------------------------------------------


In [4]:
# ===== CÉLULA 3: Inspeção Básica dos DataFrames (Resumida) =====
if dataframes_loaded: # Prosseguir apenas se os DFs principais foram carregados
    print("\n\n--- Inspeção Resumida dos DataFrames ---")
    if not df_cosif.empty:
        print(f"\n-- df_cosif_final ({df_cosif.shape}) --")
        display(df_cosif.head(2))
        if 'COD_CONGL' in df_cosif.columns: print(f"  COD_CONGL nulos: {df_cosif['COD_CONGL'].isnull().sum()}")
    if not df_ifdata.empty:
        print(f"\n-- df_ifdata_final ({df_ifdata.shape}) --")
        display(df_ifdata.head(2))
        if 'NOME_INSTITUICAO_IF' in df_ifdata.columns: print(f"  NOME_INSTITUICAO_IF nulos: {df_ifdata['NOME_INSTITUICAO_IF'].isnull().sum()}")
        if 'CNPJ_ENTIDADE_IF_BASE8' in df_ifdata.columns: print(f"  CNPJ_ENTIDADE_IF_BASE8 nulos: {df_ifdata['CNPJ_ENTIDADE_IF_BASE8'].isnull().sum()}")
    if not df_map_prud.empty:
        print(f"\n-- df_mapeamento_prudencial_geral ({df_map_prud.shape}) --")
        display(df_map_prud.head(2))
    if not df_if_cadastro_proc.empty:
        print(f"\n-- df_if_cadastro_completo_processado ({df_if_cadastro_proc.shape}) --")
        display(df_if_cadastro_proc.head(2))
        if 'CNPJ_ENTIDADE_IF_BASE8' in df_if_cadastro_proc.columns:
            print(f"  CNPJ_ENTIDADE_IF_BASE8 nulos (cadastro completo): {df_if_cadastro_proc['CNPJ_ENTIDADE_IF_BASE8'].isnull().sum()}")
            print(f"  CNPJ_ENTIDADE_IF_BASE8 únicos (cadastro completo): {df_if_cadastro_proc['CNPJ_ENTIDADE_IF_BASE8'].nunique(dropna=False)}")

    print("-" * 50)



--- Inspeção Resumida dos DataFrames ---

-- df_cosif_final ((224757, 11)) --


,DATA,DOCUMENTO,CNPJ,AGENCIA,NOME_INSTITUICAO,TAXONOMIA,CONTA_COSIF,NOME_CONTA_COSIF,SALDO_COSIF,COD_CONGL,NOME_CONGL
0,202401,4010,00000000,NaN,BCO DO BRASIL S.A.,BANCO DO BRASIL - BANCO MULTIPLO,10000007,ATIVO REALIZÁVEL,2.122392e+12,C0080329,BB - PRUDENCIAL
1,202401,4010,00000000,NaN,BCO DO BRASIL S.A.,BANCO DO BRASIL - BANCO MULTIPLO,11000006,DISPONIBILIDADES,1.633250e+10,C0080329,BB - PRUDENCIAL


  COD_CONGL nulos: 104779

-- df_ifdata_final ((2907128, 27)) --


,TipoInstituicao,COD_CONGL,DATA,NomeRelatorio,NumeroRelatorio,Grupo,CONTA_IF,NOME_CONTA_IF,DescricaoColuna,SALDO_IF,...,TC_IF,SEGMENTO_TB_IF,ATIVIDADE_IF,UF_IF,MUNICIPIO_IF,SR_IF,COD_CONGL_FINANCEIRO_IF,DATA_INICIO_ATIVIDADE_IF,SITUACAO_IF,CNPJ_LIDER_BASE8_IF
0,1,07669921,202403,Segmentação,6,nan,85845,Instituição Utiliza Metodologia Simplificada,Informação se a instituição utiliza a metodolo...,1.00,...,2.0,Cooperativa de Crédito,Atípicas,SP,São Carlos,S5,None,200511.0,A,00000000
1,1,07669921,202403,Segmentação,6,nan,86846,Exposição Total ou Ativo Total (R$ mil),Para cálculo do porte da instituição é conside...,69445430.77,...,2.0,Cooperativa de Crédito,Atípicas,SP,São Carlos,S5,None,200511.0,A,00000000


  NOME_INSTITUICAO_IF nulos: 1151844
  CNPJ_ENTIDADE_IF_BASE8 nulos: 1244341

-- df_mapeamento_prudencial_geral ((162, 3)) --


,CNPJ,COD_CONGL,NOME_CONGL
0,00000000,C0080329,BB - PRUDENCIAL
1,00000208,C0080288,BRB - PRUDENCIAL



-- df_if_cadastro_completo_processado ((65388, 18)) --


,COD_INST_OU_CNPJ_IF,DATA,NomeInstituicao,DataInicioAtividade,Tcb,Td,Tc,SegmentoTb,Atividade,Uf,Municipio,Sr,CodConglomeradoFinanceiro,CnpjInstituicaoLider,Situacao,CNPJ_ENTIDADE_IF_BASE8,COD_INST_ORIGINAL_IF,COD_CONGL
0,C0041856,202401,ABC-BRASIL,180001,B1,C,3,nan,Crédito Atacado,SP,São Paulo,S3,C0041856,28195667.0,A,None,C0041856,C0080312
1,C0080312,202401,ABC-BRASIL - PRUDENCIAL,180001,B1,I,3,nan,Crédito Atacado,SP,São Paulo,S3,C0041856,28195667.0,A,None,C0080312,C0080312


  CNPJ_ENTIDADE_IF_BASE8 nulos (cadastro completo): 8006
  CNPJ_ENTIDADE_IF_BASE8 únicos (cadastro completo): 4882
--------------------------------------------------


In [5]:
# ===== CÉLULA 4: Verificação Específica para Nubank em df_ifdata_final e df_if_cadastro_completo_processado =====
if dataframes_loaded:
    print("\n--- Verificando df_ifdata_final e df_if_cadastro_completo_processado para CNPJ do Nubank (30680829) ---")
    cnpj_nubank_interesse = '30680829'
    cod_congl_nubank_esperado = 'C0084693'

    # Verificar em df_ifdata_final (o que é usado pela função de consulta)
    if not df_ifdata.empty and 'CNPJ_ENTIDADE_IF_BASE8' in df_ifdata.columns:
        print("\n-- Em df_ifdata_final:")
        nubank_no_ifdata_final = df_ifdata[df_ifdata['CNPJ_ENTIDADE_IF_BASE8'] == cnpj_nubank_interesse]
        if not nubank_no_ifdata_final.empty:
            print(f"  >>> SUCESSO: CNPJ {cnpj_nubank_interesse} ENCONTRADO em df_ifdata_final['CNPJ_ENTIDADE_IF_BASE8']!")
            cols_to_show = ['DATA', 'COD_CONGL', 'NOME_INSTITUICAO_IF', 'CNPJ_LIDER_BASE8_IF', 'CNPJ_ENTIDADE_IF_BASE8']
            display(nubank_no_ifdata_final[cols_to_show].drop_duplicates().head())
        else:
            print(f"  >>> FALHA: CNPJ {cnpj_nubank_interesse} NÃO ENCONTRADO em df_ifdata_final['CNPJ_ENTIDADE_IF_BASE8'].")
    else:
        print("  df_ifdata_final está vazio ou não possui 'CNPJ_ENTIDADE_IF_BASE8'.")

    # Verificar em df_if_cadastro_completo_processado (a fonte da informação cadastral)
    if not df_if_cadastro_proc.empty and 'CNPJ_ENTIDADE_IF_BASE8' in df_if_cadastro_proc.columns:
        print("\n-- Em df_if_cadastro_completo_processado:")
        nubank_no_cadastro_proc = df_if_cadastro_proc[df_if_cadastro_proc['CNPJ_ENTIDADE_IF_BASE8'] == cnpj_nubank_interesse]
        if not nubank_no_cadastro_proc.empty:
            print(f"  >>> INFO: CNPJ {cnpj_nubank_interesse} ENCONTRADO em df_if_cadastro_completo_processado['CNPJ_ENTIDADE_IF_BASE8']!")
            cols_to_show_cad = ['DATA', 'COD_CONGL', 'NomeInstituicao', 'CnpjInstituicaoLider', 'CNPJ_ENTIDADE_IF_BASE8', 'COD_INST_OU_CNPJ_IF']
            display(nubank_no_cadastro_proc[cols_to_show_cad].drop_duplicates().head())
            
            # Verificar se o COD_CONGL associado é o esperado
            if cod_congl_nubank_esperado in nubank_no_cadastro_proc['COD_CONGL'].unique():
                print(f"    E está associado ao COD_CONGL esperado: {cod_congl_nubank_esperado}")
            else:
                print(f"    ALERTA: Mas NÃO está associado ao COD_CONGL esperado {cod_congl_nubank_esperado}. COD_CONGLs encontrados: {nubank_no_cadastro_proc['COD_CONGL'].unique()}")
        else:
            print(f"  >>> INFO: CNPJ {cnpj_nubank_interesse} NÃO ENCONTRADO em df_if_cadastro_completo_processado['CNPJ_ENTIDADE_IF_BASE8'].")
    else:
        print("  df_if_cadastro_completo_processado está vazio ou não possui 'CNPJ_ENTIDADE_IF_BASE8'.")
    print("-" * 50)


--- Verificando df_ifdata_final e df_if_cadastro_completo_processado para CNPJ do Nubank (30680829) ---

-- Em df_ifdata_final:
  >>> FALHA: CNPJ 30680829 NÃO ENCONTRADO em df_ifdata_final['CNPJ_ENTIDADE_IF_BASE8'].

-- Em df_if_cadastro_completo_processado:
  >>> INFO: CNPJ 30680829 ENCONTRADO em df_if_cadastro_completo_processado['CNPJ_ENTIDADE_IF_BASE8']!


,DATA,COD_CONGL,NomeInstituicao,CnpjInstituicaoLider,CNPJ_ENTIDADE_IF_BASE8,COD_INST_OU_CNPJ_IF
3526,202401,C0084693,"NU FINANCEIRA S.A. - SOCIEDADE DE CRÉDITO, FIN...",18236120.0,30680829,30680829
8927,202402,C0084693,"NU FINANCEIRA S.A. - SOCIEDADE DE CRÉDITO, FIN...",18236120.0,30680829,30680829
14324,202403,C0084693,"NU FINANCEIRA S.A. - SOCIEDADE DE CRÉDITO, FIN...",18236120.0,30680829,30680829
19732,202404,C0084693,"NU FINANCEIRA S.A. - SOCIEDADE DE CRÉDITO, FIN...",18236120.0,30680829,30680829
25149,202405,C0084693,"NU FINANCEIRA S.A. - SOCIEDADE DE CRÉDITO, FIN...",18236120.0,30680829,30680829


    E está associado ao COD_CONGL esperado: C0084693
--------------------------------------------------


In [6]:
# ===== CÉLULA 5: Chamadas à Função de Consulta =====
if dataframes_loaded and hasattr(du, 'consultar_indicador_banco'):
    print("\n\n--- Consultas de Indicadores ---")

# Em DataAnalysis2.ipynb, Célula 5, ANTES de chamar consultar_indicador_banco

    print("\n--- Verificando colunas de df_ifdata ANTES da consulta ---")
    if not df_ifdata.empty:
        print(df_ifdata.columns.tolist())
        # Verificar especificamente se 'DATA' existe e seu tipo
        if 'DATA' in df_ifdata.columns:
            print(f"Coluna 'DATA' existe em df_ifdata. Dtype: {df_ifdata['DATA'].dtype}")
            print(f"Amostra de DATA: {df_ifdata['DATA'].head().tolist()}")
        else:
            print("ALERTA CRÍTICO: Coluna 'DATA' NÃO encontrada em df_ifdata!")
    else:
        print("df_ifdata está vazio.")
    print("-" * 50)

# resultado_inter = du.consultar_indicador_banco(...)
    
    # Exemplo para o Banco Inter
    resultado_inter = du.consultar_indicador_banco(
        identificador_banco='00416968',
        data_alvo=202412, # Certifique-se que esta data tem dados IFDATA (fim de trimestre)
        indicador_nome_cosif='PATRIMONIO LIQUIDO', # Use nome exato do dicionario_contas_cosif.xlsx
        indicador_nome_ifdata='Índice de Basileia', # Use nome exato do dicionario_contas_ifdata.xlsx
        df_banks=df_banks,
        df_cosif=df_cosif,
        df_ifdata=df_ifdata,
        df_map_prud_geral=df_map_prud, # Passando o mapeamento geral
        df_if_cadastro_completo=df_if_cadastro_proc # Passando o cadastro completo processado
    )
    if resultado_inter:
        print("\nResultado da Consulta para Inter:")
        for chave, valor in resultado_inter.items():
            if chave not in ['info_entidade']:
                 print(f"  {chave}: {valor}")

    # Exemplo para Nubank
    nome_nubank_em_df_banks = "NUBANK" # <<< AJUSTE ESTE NOME PARA COMO ESTÁ EM SEU df_banks['NOME'] EXATO
    
    print(f"\nTentando consulta para: '{nome_nubank_em_df_banks}'")
    resultado_nubank = du.consultar_indicador_banco(
        identificador_banco=nome_nubank_em_df_banks, 
        data_alvo=202412, 
        indicador_nome_cosif='PATRIMONIO LIQUIDO', 
        indicador_nome_ifdata='Índice de Basileia', # Teste com uma métrica comum
        df_banks=df_banks,
        df_cosif=df_cosif,
        df_ifdata=df_ifdata,
        df_map_prud_geral=df_map_prud,
        df_if_cadastro_completo=df_if_cadastro_proc
    )
    if resultado_nubank:
        print(f"\nResultado da Consulta para {nome_nubank_em_df_banks}:")
        for chave, valor in resultado_nubank.items():
            if chave not in ['info_entidade']:
                print(f"  {chave}: {valor}")
    else:
        print(f"Consulta para {nome_nubank_em_df_banks} não retornou resultados (ou banco não encontrado).")

    print("-" * 50)
else:
    print("Um ou mais DataFrames não foram carregados, ou 'du.consultar_indicador_banco' não está disponível. Pulando consultas.")



--- Consultas de Indicadores ---

--- Verificando colunas de df_ifdata ANTES da consulta ---
['TipoInstituicao', 'COD_CONGL', 'DATA', 'NomeRelatorio', 'NumeroRelatorio', 'Grupo', 'CONTA_IF', 'NOME_CONTA_IF', 'DescricaoColuna', 'SALDO_IF', 'DATA_ULTIMO_CADASTRO_IF', 'NOME_INSTITUICAO_IF', 'CNPJ_LIDER_IF14', 'CNPJ_ENTIDADE_IF_BASE8', 'COD_INST_ORIGINAL_IF', 'TCB_IF', 'TD_IF', 'TC_IF', 'SEGMENTO_TB_IF', 'ATIVIDADE_IF', 'UF_IF', 'MUNICIPIO_IF', 'SR_IF', 'COD_CONGL_FINANCEIRO_IF', 'DATA_INICIO_ATIVIDADE_IF', 'SITUACAO_IF', 'CNPJ_LIDER_BASE8_IF']
Coluna 'DATA' existe em df_ifdata. Dtype: Int64
Amostra de DATA: [202403, 202403, 202403, 202403, 202403]
--------------------------------------------------

--- Consultando Dados para '00416968' (CNPJ de Interesse: 00416968) na Data: 202412 ---
Informações da Entidade Encontradas: {'cnpj_interesse': '00416968', 'nome_interesse': 'INTER', 'cod_congl': 'C0080996', 'cnpj_reporte_cosif_prud': '00416968', 'nome_congl_prud': 'INTER - PRUDENCIAL', 'nome

In [7]:
# ===== CÉLULA 6: Outras Análises e Conclusões (Adapte do seu notebook anterior) =====
if dataframes_loaded:
    print("\n\n--- Análise de Cobertura dos Bancos de Interesse (df_banks) ---")
    # Reutilize ou adapte sua Célula 7 anterior aqui para verificar cobertura.
    # Exemplo:
    cnpjs_interesse = df_banks['CNPJ'].tolist()
    bancos_no_cosif_count = df_cosif[df_cosif['CNPJ'].isin(cnpjs_interesse)]['CNPJ'].nunique()
    print(f"Bancos de interesse com dados no COSIF: {bancos_no_cosif_count} de {len(cnpjs_interesse)}")

    # Para IFDATA, a ligação é mais complexa.
    # Primeiro, veja quantos CNPJs de interesse têm um COD_CONGL via df_map_prud
    df_banks_com_mapa = pd.merge(df_banks, df_map_prud, on='CNPJ', how='left')
    cod_congls_interesse_via_mapa = df_banks_com_mapa['COD_CONGL'].dropna().unique()
    bancos_com_ifdata_via_mapa_cosif = df_ifdata[df_ifdata['COD_CONGL'].isin(cod_congls_interesse_via_mapa)]['COD_CONGL'].nunique()
    print(f"Bancos de interesse com dados IFDATA (via CNPJ->COD_CONGL do COSIF Prudencial): {bancos_com_ifdata_via_mapa_cosif} (contagem de COD_CONGLs)")

    # Segundo, veja quantos CNPJs de interesse aparecem como CNPJ_ENTIDADE_IF_BASE8 em df_if_cadastro_proc
    # e se esses têm COD_CONGL com dados em df_ifdata
    if not df_if_cadastro_proc.empty and 'CNPJ_ENTIDADE_IF_BASE8' in df_if_cadastro_proc.columns:
        entidades_interesse_no_cadastro_if = df_if_cadastro_proc[df_if_cadastro_proc['CNPJ_ENTIDADE_IF_BASE8'].isin(cnpjs_interesse)]
        cod_congls_interesse_via_cadastro_if = entidades_interesse_no_cadastro_if['COD_CONGL'].dropna().unique()
        bancos_com_ifdata_via_entidade_if = df_ifdata[df_ifdata['COD_CONGL'].isin(cod_congls_interesse_via_cadastro_if)]['COD_CONGL'].nunique()
        print(f"Bancos de interesse com dados IFDATA (via CNPJ->COD_CONGL do IFDATA Cadastro): {bancos_com_ifdata_via_entidade_if} (contagem de COD_CONGLs)")

    print("-" * 50)


    print("\n\n--- Conclusões Preliminares e Próximos Passos ---")
    print("""
    1. Verifique os outputs das Células 3 e 4:
       - A coluna 'CNPJ_ENTIDADE_IF_BASE8' em 'df_ifdata_final' está sendo populada para o Nubank?
       - A coluna 'CNPJ_ENTIDADE_IF_BASE8' em 'df_if_cadastro_completo_processado' contém '30680829'?
    2. Se a consulta para o Nubank na Célula 5 ainda falhar em encontrar o 'cod_congl':
       - Adicione prints de depuração DENTRO da função 'encontrar_info_entidade' em 'DataUtils.py'
         para rastrear como 'cnpj_interesse_base8="30680829"' é processado.
         Foco nos blocos 1b e 1c dessa função.
    3. Nomes de Contas/Métricas: Para os exemplos, use nomes EXATOS dos seus arquivos de dicionário
       (`dicionario_contas_cosif.xlsx`, `dicionario_contas_ifdata.xlsx`).
    """)
else:
    print("Análises puladas pois um ou mais DataFrames não foram carregados.")



--- Análise de Cobertura dos Bancos de Interesse (df_banks) ---
Bancos de interesse com dados no COSIF: 35 de 42
Bancos de interesse com dados IFDATA (via CNPJ->COD_CONGL do COSIF Prudencial): 30 (contagem de COD_CONGLs)
Bancos de interesse com dados IFDATA (via CNPJ->COD_CONGL do IFDATA Cadastro): 41 (contagem de COD_CONGLs)
--------------------------------------------------


--- Conclusões Preliminares e Próximos Passos ---

    1. Verifique os outputs das Células 3 e 4:
       - A coluna 'CNPJ_ENTIDADE_IF_BASE8' em 'df_ifdata_final' está sendo populada para o Nubank?
       - A coluna 'CNPJ_ENTIDADE_IF_BASE8' em 'df_if_cadastro_completo_processado' contém '30680829'?
    2. Se a consulta para o Nubank na Célula 5 ainda falhar em encontrar o 'cod_congl':
       - Adicione prints de depuração DENTRO da função 'encontrar_info_entidade' em 'DataUtils.py'
         para rastrear como 'cnpj_interesse_base8="30680829"' é processado.
         Foco nos blocos 1b e 1c dessa função.
    3. N

In [8]:
# ===== CÉLULA NOVA: Tabela Resumo para Bancos de Interesse (COSIF busca por CNPJ_INTERESSE) =====

if dataframes_loaded and hasattr(du, 'consultar_indicador_banco') and not df_banks.empty:
    print("\n\n--- Gerando Tabela Resumo de Indicadores para Bancos em df_banks ---")
    
    data_alvo_resumo = 202412 # Defina a data para a qual você quer os indicadores
    
    # Defina os indicadores que você quer buscar.
    # Use os nomes EXATOS das contas/métricas dos seus dicionários .xlsx
    indicadores_para_buscar = [
        {'tipo': 'COSIF', 'nome_bruto': 'PATRIMÔNIO LÍQUIDO', 'nome_coluna': 'PL_COSIF'},
        {'tipo': 'COSIF', 'nome_bruto': 'TOTAL GERAL DO ATIVO', 'nome_coluna': 'Ativo_Total_COSIF'},
        {'tipo': 'COSIF', 'nome_bruto': 'OPERAÇÕES DE CRÉDITO', 'nome_coluna': 'Oper_Credito_COSIF'},
        {'tipo': 'COSIF', 'nome_bruto': 'RESULTADO CREDOR', 'nome_coluna': 'Lucro_Liquido_COSIF'},
        {'tipo': 'IFDATA', 'nome_bruto': 'Índice de Basileia', 'nome_coluna': 'Indice_Basileia_IF'},
        {'tipo': 'IFDATA', 'nome_bruto': 'Ativos Ponderados pelo Risco (RWA) (j) = (f) + (g) + (h) + (i)', 'nome_coluna': 'RWA_IF'},
        {'tipo': 'IFDATA', 'nome_bruto': 'Patrimônio de Referência para Comparação com o RWA', 'nome_coluna': 'PR_IF'},
        {'tipo': 'IFDATA', 'nome_bruto': 'Índice de Imobilização', 'nome_coluna': 'Indice_Imob_IF'}
    ]
    
    lista_resultados_bancos = []

    for index, row in df_banks.iterrows():
        cnpj_banco_interesse = row['CNPJ'] # CNPJ da sua lista df_banks (já padronizado)
        nome_banco = row['NOME']
        print(f"\nProcessando: {nome_banco} (CNPJ Interesse: {cnpj_banco_interesse}) para {data_alvo_resumo}...")
        
        dados_banco_atual = {
            'NOME_BANCO': nome_banco,
            'CNPJ_INTERESSE': cnpj_banco_interesse
        }
        
        # Obter informações da entidade para mapeamento (principalmente para COD_CONGL para IFDATA)
        info_entidade = du.encontrar_info_entidade(
            cnpj_interesse_base8=cnpj_banco_interesse,
            df_banks=df_banks,
            df_map_prud_geral=df_map_prud,
            df_ifdata_final_valores=df_ifdata,
            df_if_cadastro_completo=df_if_cadastro_proc
        )
        
        dados_banco_atual['COD_CONGL_ENCONTRADO'] = info_entidade.get('cod_congl')
        dados_banco_atual['NOME_INST_IF_ENCONTRADO'] = info_entidade.get('nome_instituicao_if')
        dados_banco_atual['CNPJ_REPORTE_COSIF_PRUD_MAPEADO'] = info_entidade.get('cnpj_reporte_cosif_prud') # Para informação


        for ind_info in indicadores_para_buscar:
            valor_indicador = None
            if ind_info['tipo'] == 'COSIF':
                # <<<< ALTERAÇÃO AQUI: Usar diretamente o CNPJ de interesse para buscar COSIF >>>>
                if cnpj_banco_interesse and cnpj_banco_interesse != "00000000":
                    valor_indicador = du.extrair_dado_cosif(
                        df_cosif=df_cosif, # df_cosif_final (baseado no COSIF Individual)
                        cnpj_reporte_cosif=cnpj_banco_interesse, # USA O CNPJ DE INTERESSE
                        data_alvo=data_alvo_resumo,
                        nome_conta_cosif=ind_info['nome_bruto']
                    )
                    # print(f"  COSIF - '{ind_info['nome_bruto']}': {valor_indicador} (usando CNPJ de interesse: {cnpj_banco_interesse})")
                # else:
                    # print(f"  COSIF - CNPJ de interesse inválido para buscar '{ind_info['nome_bruto']}'.")

            elif ind_info['tipo'] == 'IFDATA':
                cod_congl_para_ifdata = info_entidade.get('cod_congl')
                if cod_congl_para_ifdata and pd.notna(cod_congl_para_ifdata) and str(cod_congl_para_ifdata).strip() not in ["00000000", "", "nan", "None"]:
                    valor_indicador = du.extrair_dado_ifdata(
                        df_ifdata=df_ifdata,
                        cod_congl_alvo=str(cod_congl_para_ifdata),
                        data_alvo=data_alvo_resumo,
                        nome_conta_if=ind_info['nome_bruto']
                    )
                    # print(f"  IFDATA - '{ind_info['nome_bruto']}': {valor_indicador} (usando COD_CONGL: {cod_congl_para_ifdata})")
                # else:
                    # print(f"  IFDATA - Não foi possível determinar COD_CONGL para '{ind_info['nome_bruto']}'.")
            
            dados_banco_atual[ind_info['nome_coluna']] = valor_indicador
        lista_resultados_bancos.append(dados_banco_atual)

    df_resumo_bancos = pd.DataFrame(lista_resultados_bancos)
    
    print("\n\n--- Tabela Resumo de Indicadores ---")
    if not df_resumo_bancos.empty:
        cols_info_banco = ['NOME_BANCO', 'CNPJ_INTERESSE', 'COD_CONGL_ENCONTRADO', 'CNPJ_REPORTE_COSIF_PRUD_MAPEADO', 'NOME_INST_IF_ENCONTRADO']
        cols_indicadores = [ind['nome_coluna'] for ind in indicadores_para_buscar]
        
        ordem_final_cols = cols_info_banco + [col for col in cols_indicadores if col in df_resumo_bancos.columns]
        # Garantir que todas as colunas estejam presentes, mesmo que não na ordem preferencial
        for col in df_resumo_bancos.columns:
            if col not in ordem_final_cols:
                ordem_final_cols.append(col)
        df_resumo_bancos = df_resumo_bancos.reindex(columns=ordem_final_cols)
        
        for col in cols_indicadores: # Formatação
            if col in df_resumo_bancos.columns and df_resumo_bancos[col].notna().any() : # Checa se há algum valor não-nulo antes de tentar formatar
                 # Tenta converter para numérico antes de formatar, tratando erros
                numeric_col = pd.to_numeric(df_resumo_bancos[col], errors='coerce')
                if "Indice" in col or "IS" in col or "BIS" in col:
                    df_resumo_bancos[col] = numeric_col.map(lambda x: f"{x:.4f}" if pd.notna(x) else None)
                else: 
                    df_resumo_bancos[col] = numeric_col.map(lambda x: f"{x:,.0f}".replace(",",".") if pd.notna(x) else None)
        display(df_resumo_bancos)
        
        try:
            path_excel_resumo = output_dir / f"resumo_bancos_indicadores_{data_alvo_resumo}.xlsx"
            df_resumo_bancos.to_excel(path_excel_resumo, index=False)
            print(f"\nTabela resumo salva em: {path_excel_resumo}")
        except Exception as e:
            print(f"Erro ao salvar tabela resumo em Excel: {e}")
    else:
        print("Nenhum resultado para gerar a tabela resumo.")
else:
    print("DataFrames essenciais não carregados ou funções de 'du' não disponíveis. Tabela resumo não gerada.")
print("-" * 50)



--- Gerando Tabela Resumo de Indicadores para Bancos em df_banks ---

Processando: ABC (CNPJ Interesse: 28195667) para 202412...

Processando: Master (CNPJ Interesse: 33923798) para 202412...

Processando: VOTORANTIM (CNPJ Interesse: 59588111) para 202412...

Processando: Banco Original (CNPJ Interesse: 92894922) para 202412...

Processando: BOCOM (CNPJ Interesse: 15114366) para 202412...

Processando: BRB (CNPJ Interesse: 00000208) para 202412...

Processando: BTG (CNPJ Interesse: 30306294) para 202412...

Processando: Agibank (CNPJ Interesse: 10664513) para 202412...

Processando: Banco C6 (CNPJ Interesse: 31872495) para 202412...

Processando: BMG (CNPJ Interesse: 61186680) para 202412...

Processando: Banco John Deere (CNPJ Interesse: 91884981) para 202412...

Processando: PINE (CNPJ Interesse: 62144175) para 202412...

Processando: INTER (CNPJ Interesse: 00416968) para 202412...

Processando: Sicoob (CNPJ Interesse: 02038232) para 202412...

Processando: Banco Fibra (CNPJ Intere

,NOME_BANCO,CNPJ_INTERESSE,COD_CONGL_ENCONTRADO,CNPJ_REPORTE_COSIF_PRUD_MAPEADO,NOME_INST_IF_ENCONTRADO,PL_COSIF,Ativo_Total_COSIF,Oper_Credito_COSIF,Lucro_Liquido_COSIF,Indice_Basileia_IF,RWA_IF,PR_IF,Indice_Imob_IF
0,ABC,28195667,C0080312,28195667,ABC BRASIL DISTRIBUIDORA DE TÍTULOS E VALORES ...,5.889.204.782,340.976.188.937,12.605.708.841,25.358.388.418,0.1648,54.387.264.573,8.961.598.205,0.0054
1,Master,33923798,C0080367,33923798,"MASTER S/A CORRETORA DE CÂMBIO, TÍTULOS E VALO...",4.173.952.336,184.510.624.606,12.752.918.381,9.304.567.464,0.1152,47.920.859.569,5.519.420.649,0.0907
2,VOTORANTIM,59588111,C0080484,59588111,ACESSO SOLUÇÕES DE PAGAMENTO S.A. - INSTITUIÇÃ...,12.724.181.834,621.762.411.339,59.048.752.133,30.334.345.335,0.1602,86.693.012.050,13.887.530.659,0.0520
3,Banco Original,92894922,C0080903,92894922,ORIGINAL - PRUDENCIAL,1.896.655.069,153.179.293.038,9.838.450.517,6.122.609.395,0.1397,20.651.217.577,2.885.432.698,0.0552
4,BOCOM,15114366,C0084624,15114366,BOCOM BBM CORRETORA DE CÂMBIO E VALORES MOBILI...,1.258.056.606,152.065.772.749,11.003.563.562,14.449.298.970,0.1472,15.140.524.943,2.228.789.698,0.0054
5,BRB,00000208,C0080288,00000208,CARTAO BRB S/A,3.543.984.545,262.002.574.730,36.131.796.389,5.028.232.513,0.1303,35.225.512.535,4.591.276.400,0.1558
6,BTG,30306294,C0080336,30306294,PAY2ALL INSTITUICAO DE PAGAMENTO LTDA,51.274.188.812,4.290.543.788.251,21.525.668.142,58.432.749.132,0.1571,456.270.599.296,71.663.405.106,0.3385
7,Agibank,10664513,C0083694,10664513,"AGIBANK FINANCEIRA S.A. - CRÉDITO, FINANCIAMEN...",2.290.991.385,104.169.143.987,22.513.525.240,4.709.532.048,0.1398,17.481.343.015,2.443.052.827,0.4028
8,Banco C6,31872495,C0084844,31872495,C6 CORRETORA DE TÍTULOS E VALORES MOBILIÁRIOS ...,2.241.587.916,2.604.601.066.951,19.354.655.327,13.618.196.516,0.1236,44.076.271.848,5.448.681.999,0.1771
9,BMG,61186680,C0080178,61186680,BMG S.A. DISTRIBUIDORA DE TÍTULOS E VALORES MO...,4.086.857.635,330.008.554.093,20.135.321.861,9.090.081.311,0.1321,28.278.822.904,3.735.761.124,0.2794



Tabela resumo salva em: ..\Output\resumo_bancos_indicadores_202412.xlsx
--------------------------------------------------
